In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations
from matplotlib import cm
from matplotlib.colors import ListedColormap
import requests
import bs4
from bs4 import BeautifulSoup
plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

#look at median income _std too
#look at quantile boundries

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_score
import xgboost as xgb
from joblib import dump, load

For the most part I am going to use a dataset that is provided by Lending club that has almost all the variables I need in a csv format- very handy. However there are a couple bits of information regarding the individual states such as average income that I will need to scrape off the internet.

sources of data
loan data-https://www.lendingclub.com/info/download-data.action
average household income-https://en.wikipedia.org/wiki/Household_income_in_the_United_States

quartile income-
https://data.census.gov/cedsci/table?q=median%20income&g=&hidePreview=true&table=S1901&tid=ACSST1Y2018.S1901&t=Income%20%28Households,%20Families,%20Individuals%29&lastDisplayedRow=16&vintage=2018&mode=

# Gathering The Data

## Lending Club Data

In [2]:
df=pd.read_csv('accepted_2007_to_2016.csv')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (18,46,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Because the dataset is so large my computer could not handle the computations required to run models properly. Therefore to reduce the amount of data I cut rows with NA values in avg_cur_bal. I chose avg_cur_bal because rows with NA values in this column usually had a high amount of NA values in other columns. If I could only work with a limited amount of data it makes sense to give myself the best quality of data possible.

In [4]:
df.loan_status.value_counts()

Current                                                783839
Fully Paid                                             396204
Charged Off                                             96554
Late (31-120 days)                                      20073
In Grace Period                                         10210
Issued                                                   6048
Late (16-30 days)                                        4555
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1615
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [5]:
df=df[np.isfinite(df['avg_cur_bal'])]

In [7]:
df['year_issued']=df.issue_d.map(lambda x : int(x.split('-')[1]))

As the Current loans are still going we should exclude them because we dont know if they are bad or good. going to consider fully paid as good. All the rest are bad.

In [8]:
df=df[df.loan_status !='Current']
df.loan_status.value_counts()

Fully Paid            339950
Charged Off            86289
Late (31-120 days)     20036
In Grace Period        10180
Issued                  6048
Late (16-30 days)       4543
Default                 1610
Name: loan_status, dtype: int64

In [9]:
df.loan_status.value_counts(normalize=True)

Fully Paid            0.725372
Charged Off           0.184120
Late (31-120 days)    0.042752
In Grace Period       0.021722
Issued                0.012905
Late (16-30 days)     0.009694
Default               0.003435
Name: loan_status, dtype: float64

## Additional supporting Data 

### Scraping Average household Salary

Below I scrape additional data from Wikepedia and downloaded data from the American cencus. As lending club provides income information I thought it may be useful to see how the loanee's income compared to their state average, and further what income band the loanee would fall under. This may be more useful than a pure income number as for example 100K in Detroit may go a lot further than 100K in San Francisco.

In [ ]:
#scraping data from tables on wikipedia
URL='https://en.wikipedia.org/wiki/Household_income_in_the_United_States'

r= requests.get(URL)
soup=BeautifulSoup(r.text,'html.parser')

state_list=[]
income_2017=[]
income_2016=[]
income_2015=[]
income_2014=[]
income_2013=[]
income_2012=[]
income_2011=[]
income_2010=[]
income_2009=[]
income_2008=[]
income_2007=[]

table= soup.find_all(attrs={'class':'wikitable'})
income_table=table[5]
for row in income_table.find_all('tr'):
    for ind,col in enumerate(row.find_all('td')):
        if ind==0 or ind==1:
            pass
        
        if ind==2:
            try:
                state_list.append(col.text)
            except:
                state_list.append(np.Nan)
                print('no')
        if ind==3:
            try:
                income_2017.append(col.text)
            except:
                income_2017.append(np.Nan)
                print('no')
        if ind==4:
            try:
                income_2016.append(col.text)
            except:
                income_2016.append(np.Nan)
                print('no')
        if ind==5:
            try:
                income_2015.append(col.text)
            except:
                income_2015.append(np.Nan)
                print('no')
                
        if ind==6:
            try:
                income_2014.append(col.text)
            except:
                income_2014.append(np.Nan)
                print('no')
        
        if ind==7:
            try:
                income_2013.append(col.text)
            except:
                income_2013.append(np.Nan)
                print('no')
        
        if ind==8:
            try:
                income_2012.append(col.text)
            except:
                income_2012.append(np.Nan)
                print('no')
                
        if ind==9:
            try:
                income_2011.append(col.text)
            except:
                income_2011.append(np.Nan)
                print('no')
        
        if ind==10:
            try:
                income_2010.append(col.text)
            except:
                income_2010.append(np.Nan)
                print('no')
                
        if ind==11:
            try:
                income_2009.append(col.text)
            except:
                income_2009.append(np.Nan)
                print('no')
        
        if ind==12:
            try:
                income_2008.append(col.text)
            except:
                income_2008.append(np.Nan)
                print('no')
                
        if ind==13:
            try:
                income_2007.append(col.text)
            except:
                income_2007.append(np.Nan)
                print('no')



In [ ]:
#creating a dataframe from the data scraped
state_info=pd.DataFrame({'state':state_list,
                            '2017': income_2017,
                           '2016': income_2016,
                           '2015':income_2015,
                           '2014':income_2014,
                           '2013':income_2013,
                           '2012':income_2012,
                           '2011':income_2011,
                           '2010':income_2010,
                           '2009':income_2009,
                           '2008':income_2008,
                         '2007':income_2007})



#cleaning the income data
for column in list(state_info.columns):
    state_info[column]=state_info[column].map(lambda x : x.rstrip('\n'))

#turning the values into integers
income_columns=list(state_info.columns)[1:]
for column in income_columns:
    state_info[column]=state_info[column].map(lambda x: int(x.replace('$','').replace(',','')))
    

#melted the dataframe so it can be merged into the original lending club dataframe
state_info=state_info.melt(id_vars='state',
               var_name='Date',
               value_name='Income')

The states in my main dataset are abbrevated so I needed to create a dictionary to convert them.

In [10]:
#found a list online of states and their abbreviations that saved me a bit of time
states_abr='''
Alabama - AL
Alaska - AK
Arizona - AZ
Arkansas - AR
California - CA
Colorado - CO
Connecticut - CT
Delaware - DE
Florida - FL
Georgia - GA
Hawaii - HI
Idaho - ID
Illinois - IL
Indiana - IN
Iowa - IA
Kansas - KS
Kentucky - KY
Louisiana - LA
Maine - ME
Maryland - MD
Massachusetts - MA
Michigan - MI
Minnesota - MN
Mississippi - MS
Missouri - MO
Montana - MT
Nebraska - NE
Nevada - NV
New Hampshire - NH
New Jersey - NJ
New Mexico - NM
New York - NY
North Carolina - NC
North Dakota - ND
Ohio - OH
Oklahoma - OK
Oregon - OR
Pennsylvania - PA
Puerto Rico - PR
Rhode Island - RI
South Carolina - SC
South Dakota - SD
Tennessee - TN
Texas - TX
Utah - UT
Vermont - VT
Virginia - VA
Washington - WA
West Virginia - WV
Wisconsin - WI
Wyoming - WY'''

#turning the string into a list
states_abr_list=states_abr.split('\n')

#turning the list into a dict
sl=states_abr_list[1:]
states_tup_list=[(x[0],x[1]) for x in [x.replace(' ','').split('-') for x in sl]]
state_dict=dict(states_tup_list)



In [11]:
#function to apply the transformation to the dataset (DC was not in the original list)
def state_to_abr(cell):
    x=cell.replace(' ','')
    try:
        return state_dict[x]
    except:
        
        
        return 'DC'

In [12]:
state_info['abreviation']=state_info.state.map(state_to_abr)


NameError: name 'state_info' is not defined

In [ ]:
#renaming the columns 
state_info.columns=['State_full','Year','average_income','abreviation']

#saving the dataset to csv just in case it gets removed from the internet
state_info.to_csv('state_info.csv')

In [ ]:
#merging the two dataframes
merged_df=pd.merge(df,state_info,left_on=['Year','addr_state'],right_on=['Year','abreviation'])
#once merged did not need the state names again so we can drop them
merged_df.drop(columns=['abreviation','State_full'],inplace=True)

In [ ]:
#checking the dataframes merged correctly
merged_df.head()

### Creating Percentile Income Bands

This is a continuation of the above thought process but on a slightly more sophisticated level.

In [ ]:
merged_df['annual_inc_joint']=merged_df.apply(lambda x: x['annual_inc'] if np.isnan(x['annual_inc_joint']) else x['annual_inc'],axis=1)

#removing the couple missing salaries
merged_df=merged_df[merged_df.annual_inc_joint.notnull()]

In [ ]:
messy_state_incomes=pd.read_csv('states_median_income.csv',index_col=0,header=2)

In [ ]:
#reformatting the columns here
estimate_columns=[]
for column in messy_state_incomes.columns:
    try:
        int(column.split('.')[1])
    except:
        estimate_columns.append(column)
non_estimate_columns=[col for col in messy_state_incomes.columns if col not in estimate_columns]

messy_state_incomes.drop(columns=non_estimate_columns,inplace=True)

In [ ]:
messy_state_incomes.dropna(inplace=True)

In [ ]:
#sorting out the income bands
percentile_state_incomes=messy_state_incomes.iloc[1:-8,:]

In [ ]:
#more reformatting to get the values into integers
percentile_state_incomes= percentile_state_incomes.applymap(lambda x: x.replace(',',''))
percentile_state_incomes=percentile_state_incomes.astype(int)

Because the salary information was not in the form of percentiles we had to convert it to percentiles using cumsum. Below is one example, and then I do it to a whole dataframe

In [ ]:
(percentile_state_incomes['Alabama']/percentile_state_incomes['Alabama'].sum()).cumsum()

In [ ]:
percentile_state_incomes
for column in percentile_state_incomes.columns:    
    percentile_state_incomes['percentile_{}'.format(column)]=(percentile_state_incomes[column]/percentile_state_incomes[column].sum()).cumsum() 

In [ ]:
#cleaning up the indexes of the dataframe to reflect the upper percentile bands and remove the unneeded punctuation

clean_list=[]
for ind in list(percentile_state_incomes.index):
    try:
        clean_list.append(int(ind.split('$')[-1].replace(',','')))
    except:
        clean_list.append(1000000)

percentile_state_incomes['upper_income_bands']=clean_list

In [ ]:
#now removing all the columns that are not percentile columns
percentile_state_incomes=percentile_state_incomes.iloc[:,-53:]
percentile_state_incomes.reset_index(inplace=True)
percentile_state_incomes.drop(columns='index',inplace=True)

In [ ]:
#melting the dataframe so we can merge it with our main dataframe on two columns- state and income band
melted_percentiles=percentile_state_incomes.melt(id_vars='upper_income_bands',
                             var_name='state',
                             value_name='percentile')

In [ ]:
#facing the same problem as above with states not being abbreviated
melted_percentiles.state= melted_percentiles.state.map(lambda x: state_to_abr(x.split('_')[1]))

In [ ]:
def find_percentile_for_income(x_row):
    #this is only for 2014 data so going to put a if statement at the top, in the future I would like to expand to other years
    if x_row['Year']=='2014':
        salary=x_row['annual_inc']
        salary_band=0
        #think splitting in the middle will mean the function will have to run less if statements
        if salary > 49999:
            if salary < 74999:
                salary_band=74999
            elif salary < 99999:
                salary_band=99999
            elif salary < 149999:
                salary_band= 149999
            elif salary < 199999:
                salary_band= 199999
            else:
                salary_band=1000000
        else:
            if salary >34999:
                salary_band=49999
            if salary > 24999:
                salary_band=34999
            if salary > 14999:
                salary_band=24999
            if salary > 10000:
                salary_band=14999
            else:
                salary_band=10000

        state=x_row['addr_state']
        return melted_percentiles[(melted_percentiles['upper_income_bands']==salary_band)&(melted_percentiles['state']==state)]['percentile'].values[0]
    
    else:
        return 0
    
#here i could have used pandas cut but had already made the function when i realised

In [ ]:
#applying to the whole dataframe
merged_df['income_percentile_for_state']=merged_df.apply(find_percentile_for_income,axis=1)

In [ ]:
#checking if there are any values set to 0
merged_df[merged_df.income_percentile_for_state==0].shape

# Cleaning the Data

In [ ]:
#getting list of missing values
merged_df.isna().sum().sort_values(ascending=False)

all_data_na=merged_df.isna().sum()/len(merged_df)*100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
all_data_na[:10]

In [ ]:
#just going to group most of the cleaning together so the code is a bit cleaner
def cleaner(data,min_list=None,max_list=None,cat_list=None,date_list=None):
    
    
    all_data_na=data.isna().sum()/len(data)*100
    all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
    print('columns with missing data before clean-\n',all_data_na)
    print('-'*20)
    
    if cat_list != None:
        
        for column in cat_list:
            data[column].fillna(' ', inplace=True)
    
    if date_list != None:
       
        for column in date_list:
            data[column]=pd.to_datetime(data[column],infer_datetime_format=True)
            
            
    if min_list!=None:
        
        for column in min_list:
            data[column].fillna((data[column].min()),inplace=True)
    
    if max_list!=None:
        
        for column in max_list:
            data[column].fillna((data[column].max()),inplace=True)
    
    #then drop dregs
    data.dropna(inplace=True)
    
    all_data_na=data.isna().sum()/len(data)*100
    all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
    print('columns with missing data after clean-\n',all_data_na)
    
    return data

In [ ]:
#did this because if only one person the joint dti is basically the normal dti
merged_df['dti_joint']=merged_df.apply(lambda x: x['dti'] if np.isnan(x['dti_joint']) else x['dti_joint'],axis=1)
#no info on this
merged_df.drop(columns='desc',inplace=True)
merged_df=merged_df[np.isfinite(merged_df['avg_cur_bal'])]

In [ ]:
#for categorical list because they were strings could leave them blank, assumed that for things like emp_title if they were blank it meant they were unemployed
cat_list=['verification_status_joint','next_pymnt_d',
          'emp_title','emp_length','title']

#for most of these I thought that if they were blank it meant that it was because they did not have/do whatever the column was asking for eg inq last 12 months being blank- no inquiries in last 12 months
min_list=['inq_last_12m','open_il_6m','inq_fi','max_bal_bc',
         'open_rv_24m','open_rv_12m','total_bal_il','open_il_12m','open_il_24m',
          'total_cu_tl','open_acc_6m','il_util','num_tl_120dpd_2m',
          'num_tl_120dpd_2m','bc_util','percent_bc_gt_75','bc_open_to_buy','mths_since_recent_bc','last_pymnt_d',
          'revol_util','pct_tl_nvr_dlq','last_credit_pull_d','all_util','mo_sin_old_il_acct'
         ]

#for max list thought that for these columns the person has not done whatever the column is looking for eg go delinqent on a loan
max_list=['mths_since_last_record','mths_since_recent_bc_dlq','mths_since_last_major_derog',
         'mths_since_recent_revol_delinq','mths_since_last_delinq','mths_since_recent_inq','mths_since_rcnt_il']

In [ ]:
merged_df=cleaner(merged_df,min_list=min_list,max_list=max_list,cat_list=cat_list,date_list=['last_credit_pull_d','last_pymnt_d','earliest_cr_line','issue_d'])

Finally to see what the distribution of fully paid/ good loans vs charged off and defaulted loans are. The way I have removed Na values has balanced the dataset somewhat which is good for my models in the short term as we will not have to employ measures such as SMOTE or undersampling. However in the future I would like to go back and include more data and therefore probably imbalance the dataset further in the hope to make my models more robust. 

In [ ]:
merged_df.loan_status.value_counts(normalize=True)

In [ ]:
merged_df.rename(columns={'average_income':'average_income_state'},inplace=True)

In [ ]:
#want to compare salary vs state average - can imagine money goes a lot further in some states vs others

merged_df['income_vs_average']=merged_df['annual_inc']-merged_df['average_income_state']

In [ ]:
#id and member id give no valuable info
#may drop next payment d too because only got 3 different values

merged_df.drop(columns=['id','member_id','next_pymnt_d','collection_recovery_fee'],inplace=True)

In [ ]:
merged_df['earliest_cr_line']=pd.to_datetime(merged_df['earliest_cr_line'],infer_datetime_format=True)
merged_df['issue_d']=pd.to_datetime(merged_df['issue_d'],infer_datetime_format=True)
merged_df['time_since first_cr_line']=merged_df['issue_d']-merged_df['earliest_cr_line']

In [ ]:
def datetime_to_float(d):
    return d.timestamp()


In [ ]:
merged_df['issue_d']=merged_df['issue_d'].map(datetime_to_float)


date_columns=['earliest_cr_line','last_pymnt_d','last_credit_pull_d']

for column in date_columns:
    merged_df[column]=merged_df[column].map(datetime_to_float)

merged_df['time_since first_cr_line']=merged_df['time_since first_cr_line'].apply(lambda x: float(x.days))

### Deciding on what loan statuses should be good vs bad

In [ ]:
#here we are deciding on our classes- what is a good loan and what is a bad loan
#think we can say that good loans are- fully paid, in grace period and issued with the rest being bad loans
good_loans=['Fully Paid','In Grace Period','Issued']

merged_df['good_or_bad']=merged_df.loan_status.map(lambda x : 1 if x not in good_loans else 0)

### Limiting the Dataset

As the Dataset was still very large and therefore any model I tried to run took far too long, I decided to focus on 2014 loans that were of a grade C or below. This is because the loans with lower grades had higher interest rates but were more likely to fail- having higher upside but also higher risk.

In [ ]:
temp_df=merged_df.copy()

temp_df=temp_df[~temp_df.Year.isin(['2007','2008','2009','2010','2012','2016','2013','2015'])]

temp_df=temp_df[temp_df.annual_inc<1000000]
#removing outliers

temp_df=temp_df[~temp_df.grade.isin(['A','B'])]

In [ ]:
temp_df.to_csv('temp_df.csv',index=False)

# NLP

Because I had some time left and I was not happy with the scores I was getting I decided to do some very simple NLP on the job titles column to see if there was any predictive power in this column. My thought process in the below code was to pick out words which appeared quite a lot in the dataset (such as manager, driver etc) and then out of that set only include them if they had any predictive power so as to reduce the complexity of the model. 

In [ ]:
#only want popular words/jobs so it will generalise well
cvec = CountVectorizer(token_pattern='\w+',min_df=10000,binary=True)

In [ ]:
#getting a count of words that appear in bad loans
bad_loans=temp_df[temp_df['good_or_bad']==1]
badcvec= CountVectorizer(token_pattern='\w+',binary=True)

In [ ]:
badwords=badcvec.fit_transform(bad_loans.emp_title)

In [ ]:
badwords.sum(axis=0)

In [ ]:
bad=pd.DataFrame(badwords.sum(axis=0),columns=badcvec.get_feature_names()).transpose().sort_values(0, ascending=False).transpose(

In [ ]:
#getting a count of words that appear in good loans
good_loans=temp_df[temp_df['good_or_bad']==0]
goodcvec= CountVectorizer(token_pattern='\w+',binary=True)

goodwords=goodcvec.fit_transform(good_loans.emp_title)

goodwords.sum(axis=0)

In [ ]:
#turning the matrix into a dataframe for ease of use
good=pd.DataFrame(goodwords.sum(axis=0),columns=goodcvec.get_feature_names()).transpose().sort_values(0, ascending=False).transpose()

In [ ]:
#turning the column names into a column 
good=good.melt(var_name='good_job',value_name='good_values')
bad=bad.melt(var_name='bad_job',value_name='bad_values')
jobs_performance=good.merge(bad,left_on='good_job',right_on='bad_job')
#finding the good loan vs bad loan 
jobs_performance['good_loan_ratio']=jobs_performance['good_values']/(jobs_performance['good_values']+jobs_performance['bad_values'])

In [ ]:
#only keeping words that appear more than 1000 times
jobs_performance=jobs_performance[(jobs_performance['good_values']+jobs_performance['bad_values'])>1000]

In [ ]:
jobs_performance.sort_values(by='good_loan_ratio',ascending=False).head()

In [ ]:
#created a function that binarizes the words with a given threshold, there is a better way of doing this out there I think
def discriminator_jobs(x):
    ''' returns job if the good to bad ratio is either 
    a lot higher than base or a lot lower than base'''
    
    if x['good_loan_ratio']>0.72 or x['good_loan_ratio']<0.62:
        return 1
    else:
        return 0    

In [ ]:
#applying the function to the dataframe
jobs_performance['discriminating_job']=jobs_performance.apply(discriminator_jobs,axis=1)

In [ ]:
discriminating_jobs=list(jobs_performance[jobs_performance.discriminating_job==1]['good_job'].values)

In [ ]:
cvec = CountVectorizer(token_pattern='\w+',vocabulary=discriminating_jobs,
                       binary=True)

In [ ]:
words=cvec.fit_transform(temp_df.emp_title)

In [ ]:
words_df=pd.DataFrame(words.toarray(),
                      columns=cvec.get_feature_names())

In [ ]:
merged_for_words=temp_df.reset_index()

In [ ]:
merged_for_words=merged_for_words.join(words_df)
merged_for_words.drop(columns=['emp_title','total_rec_prncp',
                      'out_prncp_inv','total_pymnt_inv','funded_amnt_inv',
                      'last_pymnt_d','out_prncp','total_pymnt','total_rec_int','last_pymnt_amnt','index'],inplace=True)

In [ ]:
merged_for_words.to_csv('merged_for_words1.csv',index=False)